In [1]:
import numpy as np
import cv2
import os
import json

In [2]:
camera_number = 1
frame_count = 0
image_number = 10

#print(os.listdir())

abs_dir_path = "videos/"
VIDEO_NAME = f'out{camera_number}F.mp4'
ROW_POINTS = 5
COLUMNS_POINTS = 7
# Open the video file 
video_capture = cv2.VideoCapture(filename=f'{abs_dir_path}{VIDEO_NAME}')

# Print the number of frames in the video
number_of_frame = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
print("Number of frames in the video: ", number_of_frame)

Number of frames in the video:  2584


In [3]:
os.listdir()
#os.chdir('..')

['videos',
 'README.md',
 '__pycache__',
 'Calibration.pdf',
 'CVprojects-Sport.pdf',
 'Rizzetto-Tomaselli_Report.pdf',
 '.git',
 'chessboard_calibration.py',
 'Accuracy_Evaluation_and_Prediction_of_Single-Image_Camera_Calibration.pdf',
 'LICENSE',
 'samples',
 'report',
 '.gitignore',
 'lib',
 'checkboard_test.ipynb',
 'appunti']

In [4]:
# Create a directory to save the screen captures named as the video file
output_dir = 'samples/' + f'out{camera_number}'
os.makedirs(output_dir, exist_ok=True)
print("Saving frames to ", output_dir)
# Remove all files in the directory
for file in os.listdir(output_dir):
    os.remove(os.path.join(output_dir, file))


Saving frames to  samples/out1


In [9]:
#findChessboard Flag test
import datetime as dt
import lib.util as util
import lib.Calibration as cal
import cv2 

one = cal.Calibration(video_path, CHESS_WIDTH, CHESS_HEIGHT)
skip = 500
img = []
print(f'starting skip of {skip}')
for i in range(skip):
    ret, img = one.video_capture.read()
    if not ret:
        break
print(f'frame after skip')

cv2.namedWindow("test image", cv2.WINDOW_NORMAL) 
  
# Using resizeWindow() 
cv2.resizeWindow("test image", 1920, 1080) 
  
# Displaying the image 
cv2.imshow("test image", img) 

one.video_capture.release()
cv2.waitKey(0)
cv2.destroyAllWindows()

#flags FAST_CHECK -> False FAST_CHECK + NORMALIZE -> False  None -> True FAST_CHECK + NORMALIZE + THRESH 
now = dt.datetime.now()
timestamp = util.getSeconds(now)
ret, imgpoints = one.extractCorners(img= img, output_dir= 'test/', find_flags=  cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE + cv2.CALIB_CB_FAST_CHECK)
now = dt.datetime.now()
t2 = util.getSeconds(now)
print(f"all flags time: {(t2 - timestamp)}")
print(ret)

now = dt.datetime.now()
timestamp = util.getSeconds(now)
#print(timestamp)
ret, imgpoints = one.extractCorners(img= img, output_dir= 'test/', find_flags=  None)
now = dt.datetime.now()
t2 = util.getSeconds(now)
#print(t2)
print(f"no flags time: {(t2 - timestamp)}")
print(ret)

##best one!
now = dt.datetime.now()
timestamp = util.getSeconds(now)
#print(timestamp)
ret, imgpoints = one.extractCorners(img= img, output_dir= 'test/', find_flags=  cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK)
now = dt.datetime.now()
t2 = util.getSeconds(now)
#print(t2)
print(f"no norm flag time: {(t2 - timestamp)}")
print(ret)

now = dt.datetime.now()
timestamp = util.getSeconds(now)
#print(timestamp)
ret, imgpoints = one.extractCorners(img= img, output_dir= 'test/', find_flags= cv2.CALIB_CB_FAST_CHECK)
now = dt.datetime.now()
t2 = util.getSeconds(now)
#print(t2)
print(f"only fast flag time: {(t2 - timestamp)}")
print(ret)


starting skip of 500
frame after skip
[[ 581.54016 1488.0059 ]]
saving images to directory
all flags time: 1435.2750000059605
True
[[ 581.54016 1488.0059 ]]
saving images to directory
no flags time: 1227.6939999982715
True
[[ 580.76636 1487.8502 ]]
saving images to directory
no norm flag time: 1193.752999998629
True
only fast flag time: 606.9409999996424
False


In [2]:
#COMPUTING INTRINSIC PARAMETERS TEST
import numpy as np
import cv2
import re
import datetime as dt
import lib.util as util
import lib.Calibration as cal
from tqdm import tqdm

camera = 'out2F'
input_dir = f'samples/{camera}'
all_chessboard_sizes = [(5, 7), (5, 7), (5, 7), (5, 7), (6, 9), (6, 9), (5, 7), (6, 9), (6, 9), (0, 0), (6, 9), (5, 7),
                        (5, 7)]
chessboard_size = all_chessboard_sizes[int(re.search('\d',camera)[0])]

CHESS_WIDTH = chessboard_size[0]
CHESS_HEIGHT = chessboard_size[1]


img_list = util.loadImagesBatch(input_dir)
img_size =  ((img_list[0]).shape[1], (img_list[0]).shape[0]) #width and height
imgpoints = []
objpoints = []

objp = np.zeros((CHESS_WIDTH * CHESS_HEIGHT, 3), np.float32)
objp[:, :2] = np.mgrid[0:CHESS_WIDTH, 0:CHESS_HEIGHT].T.reshape(-1, 2)


for img in tqdm(img_list):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, points = cv2.findChessboardCorners(gray,(CHESS_WIDTH, CHESS_HEIGHT),None, cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_ADAPTIVE_THRESH)
    if ret:
        objpoints.append(objp)
        imgpoints.append(points)

#no subpixel cases

    #tc std.
tc = (cv2.TERM_CRITERIA_EPS, 0.005)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size , None, tc)
json_camera_matrix = {
        'ret' : ret,
        'mtx' : mtx.tolist(),
        'dist': dist.tolist(),
        'rvecs' : [rvecs[0].tolist(), rvecs[1].tolist()],
        'tvecs' : [tvecs[0].tolist(), tvecs[1].tolist()]
}
util.saveToJSON(json_camera_matrix, f'{camera}std')
    #tc high accuracy
tc = (cv2.TERM_CRITERIA_EPS, 0.00001)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size , None, tc)
json_camera_matrix = {
        'ret' : ret,
        'mtx' : mtx.tolist(),
        'dist': dist.tolist(),
        'rvecs' : [rvecs[0].tolist(), rvecs[1].tolist()],
        'tvecs' : [tvecs[0].tolist(), tvecs[1].tolist()]
}
util.saveToJSON(json_camera_matrix, f'{camera}high_acc')
    #tc high accuracy and iteration
tc = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 0.005, 50)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size , None, tc)
json_camera_matrix = {
        'ret' : ret,
        'mtx' : mtx.tolist(),
        'dist': dist.tolist(),
        'rvecs' : [rvecs[0].tolist(), rvecs[1].tolist()],
        'tvecs' : [tvecs[0].tolist(), tvecs[1].tolist()]
}
util.saveToJSON(json_camera_matrix, f'{camera}iter')

#subpixel refinement



100%|██████████| 60/60 [01:00<00:00,  1.00s/it]


True

In [3]:
#UNDISTORTION TEST
import numpy as np
import cv2
import os
import re
import datetime as dt
import lib.util as util
import lib.Calibration as cal

camera = 'out2F'
image = 'chessboard55'

und_dir = f'undistorted/{camera}'
os.makedirs(und_dir, exist_ok=True)

filepath = f'samples/{camera}/{image}.jpg'
img = cv2.imread(filename=filepath)

#showing original image
util.showImage(img, 'original')

#refining camera matrix
dict = util.LoadJSON('json/out2Fhigh_acccorners.json')
h, w = img.shape[:2]
#standard values, taken from the json
std_mtx = np.array(dict['mtx'])
std_dist = np.array(dict['dist'])
print(f"numpy matrix: {std_mtx}")
new_mtx, roi = cv2.getOptimalNewCameraMatrix(std_mtx, std_dist, (w,h), 1, (w,h))

x, y, w, h = roi

#undistort the image, corrected with the new computed camera matrix
new_dist = np.array([])
und_img = cv2.undistort(src=img, cameraMatrix= std_mtx, dst= new_dist ,distCoeffs= std_dist, newCameraMatrix= new_mtx)
util.showImage(und_img, 'undistorted - new camera matrix - no crop')
cv2.imwrite(f'{und_dir}/{camera}_new_mtx_no.jpg',und_img)
#print((np.size(np.array(und_img),0),np.size(np.array(und_img),1)))
# crop
crop_und_img = und_img[y:y+h, x:x+w]
util.showImage(crop_und_img, 'undistorted - new camera matrix - cropped')
cv2.imwrite(f'{und_dir}/{camera}_new_mtx.jpg', crop_und_img)
#no new camera matrix
und_img = cv2.undistort(src=img, cameraMatrix= std_mtx, dst= new_dist ,distCoeffs= std_dist)
util.showImage(und_img, 'undistorted - standard camera matrix - no crop')
cv2.imwrite(f'{und_dir}/{camera}_std_mtx.jpg',und_img)
# crop 
# no_crop_und_img = und_img[y:y+h, x:x+w]
#util.showImage(und_img, 'undistorted - standard camera matrix - cropped')


numpy matrix: [[2.04228338e+03 0.00000000e+00 1.96746787e+03]
 [0.00000000e+00 2.04802155e+03 1.17392187e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


True

In [10]:
##JSON TEST
frame_skip = 10
for i in range(1400):
    ret, img = video_capture.read()

json_dict = {}

while True:
        # Read a frame from the video
        ret, img = video_capture.read()
        if not ret:
            break  # Break the loop if we've reached the end of the video

        # Skip frames based on the frame_skip value
        if frame_count % frame_skip == 0 :
            # Resize the frame to the new_resolution
            real_img = img
            
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            #corners = []
            # Find the chessboard corners
            found_corners, corners = cv2.findChessboardCorners(gray, (ROW_POINTS, COLUMNS_POINTS), corners= None)
            #res_img = cv2.drawChessboardCorners(img,(ROW_POINTS,COLUMNS_POINTS),corners,found_corners)
            if found_corners:
                #populating the dictionary with the points
                print('saving to JSON')
                json_dict.update([[f"{camera_number}_{1000 + frame_count}" , corners.tolist()]])
            print(frame_count)
        
            #print("Frame ", frame_count, ":", found_corners)
            #res_img = cv2.cvtColor(res_img, cv2.COLOR_GRAY2BGR)
        frame_count += 1  
        if frame_count == 300:
            break  

res = saveToJSON(json_dict, camera_number)
print(res)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
saving to JSON
140
150
160
170
180
190
200
210
220
saving to JSON
230
240
250
260
270
280
290
True


In [22]:
saveToJSON(np.array([[12,3],[3,3]]),1,210)
saveToJSON(np.array([[12,3],[3,3]]),2,100)
